In [1]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import Adam
import torch.nn.functional as F

In [2]:
!git clone https://github.com/e9t/nsmc.git

fatal: destination path 'nsmc' already exists and is not an empty directory.


In [3]:
train_df = pd.read_csv('./nsmc/ratings_train.txt', sep='\t')
test_df = pd.read_csv('./nsmc/ratings_train.txt', sep='\t')

In [4]:
train_df

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [5]:
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

train_df = train_df.sample(frac=0.2, random_state=999)
test_df = test_df.sample(frac=0.2, random_state=999)

In [6]:
train_df

,id,document,label
103096,1269448,이 영화(제작과정포함)를 접한 후 결론 → 샤론스톤은 쓰레기다.,0
4038,9867479,넘 멋져열. 몇번씩 보게되는~~,1
135345,9924513,호러 공포영화 정말 좋아해서 거의 다 봤어요. 최근 본 공포물중 최고입니다. 쫄깃쫄...,1
119472,7074014,웃으면서 잘 봤네요,1
64192,8359338,어음...이거 다 보고 딱 생각난대사가 ...뭐지...? 왜 이렇게 갑자기 끝나? ...,0
...,...,...,...
133103,918379,브로스넌판 007 최고작품,1
7855,163139,웃겨서 죽는줄 알았다,1
148395,5529691,은행털이소재좋아하는데 이영화는 시간날리기딱좋은영화,0
36882,578958,간디 그의 일생자체만으로도!!,1


In [7]:
train_df.iloc[0, 2]

0

In [8]:
class NsmcDataset(Dataset):
    ''' Naver Sentiment Movie Corpus Dataset '''
    def __init__(self, df): #데이터 전처리
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.iloc[idx, 1]
        label = self.df.iloc[idx, 2]
        return text, label

In [9]:
nsmc_train_dataset = NsmcDataset(train_df)
train_loader = DataLoader(nsmc_train_dataset, batch_size=2, shuffle=True, num_workers=2)


In [10]:
nsmc_train_dataset

In [11]:
nsmc_train_dataset.__getitem__(3)

('웃으면서 잘 봤네요', 1)

In [12]:
train_loader

In [13]:
import torch

#check whether cuda is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  

#device = torch.device("cuda")
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-cased')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [14]:
for text, label in train_loader :
    print("text:",text)
    print("label:",label)

text: ('지금봐도 전혀 촌스럽지 않는 스토리와 짐 캐리의 연기 그리고 카메론 디아즈의 외모', '너무재밋게보고있어요부모님이랑같이보는데 빵빵터지고 눈물샘도자극하네요..너무좋은프로그램이네요..^^여군화이팅!김수로상병 서경석상병 샘상병 조교넘멋져요~')
label: tensor([1, 1])
text: ('ㅉ 에휴... 범인은 무슨 부모님한테 배웠길래 저런 쓰레기 짓만 해서.. 에휴... 좋겠다 부모님이 그렇게가르쳐줘서', '좀비영화계의 레전드..좀비들이 정말미친듯이달려오는게 진짜무서웠음')
label: tensor([1, 1])
text: ('아이들 나오는 프로는 애들을 너무 좋아해서 빼놓지 않고 다보는 사람입니다. 제가 붕어빵을 틀게되도 바로 채널을 돌려버리는 이유는 아이들이 대본을 외운듯 말하네요.참 부자연스럽고 아이들만의 순수하고 엉뚱한 모습이 없어서 아쉽습니다.', '모든 음악이 훌륭했지만 트롤들을 만났을 때의 음악은 정말 불후의 명곡이다. 애니 음악 하나에도 이렇게 최고의 음악을 만드는 그들의 문화 수준...대단해.')
label: tensor([0, 1])
text: ('그러니까 뭔가 괜찮았단 말이지...뭔가 괜찮아서 몰입했는데 뭐였는지는 모르겠다는 것이지...짜릿한 맛은 전혀 없는데 그 말하기 힘든 뭔가가 매우 괜찮았음...', '영화같은 아름다운 삶을 표현한 영화')
label: tensor([1, 1])
text: ('이거 막 나왓을땐 진짜 재밋엇음 지금 봐도 체감공포는 최곤데..', '솔직히 오현서 때문에 돈내고 봤는대요.. 정말재미없내요.. 몰입이 안됩니다..연기들이 너무 부자연스럽고 내용전개가 안되요 밴드드럼치는사람..그노래작곡가? 무슨 교과서 읽는줄알앗습니다..')
label: tensor([1, 0])
text: ('그 때로 돌아가고 싶어진다.', '뭐 하나 빠짐없이 부족하다. 연기 비쥬얼 연출 음악 스토리 모두 볼품없다. 구지 평 하자면 사극판 장미의 전쟁을 본것 같다. 주인 없는 꽃 같은 영화다. 아무도 주인이 되려 하지 않을것.

label: tensor([1, 0])
text: ('제눈에는 원수를사랑하라! 원수를 갑거나 진노를 내리는건 하나님의 역활이다!라고 성경에 나와있습니다.이목사님은 특별하게 하나님의 진노의 집행자로 즉 사자로써 유일하게 쓰시는게 아닐까 합니다.', '이 영화는 감히 그 어떤 명감독들도 시도하지 못할 21세기 대한민국의 위대한 문화유산입니다~아마 다음세기가 오기까지는 21세기 대작명화는 이 영화가 유일하다고 장담해요.죽기전에 꼭 한번 봐야할 명화입니다! 아들딸 손자손녀까지 두고두고 회자될 영화~~')
label: tensor([1, 1])
text: ('평점이 너무 낮네요...', '그냥 실없는 영화')
label: tensor([1, 0])
text: ('그냥 영화 좀 재미없고 어렵게만 만들면 다 예술영화?ㅋㅋ 소름은 게뿔 닭살아니냐?', '다큐멘터리의 재미와 상관없이 메시빠들이 일부러 최하점주는거보니까 나는 위대한선수에 최고점을 주고싶다')
label: tensor([0, 1])
text: ('객관적으로 재미가없음', '와...OO 괴물이 연기 더잘하더라...')
label: tensor([0, 0])
text: ('이 드라마에서 정상인 사람은 한사장이랑 윤재아버지뿐인듯...더는 봐줄수가 없어서 오자룡으로 채널 돌렸습니다. 콩가루집안에, 찌질한 남자주인공에, 이해할수 없는 여자주인공, 기타등등...안보는게 정신건강에 좋겠어요.', '눈물흘리면서 봤습니다. 그리스도인들에게 추천합니다!')
label: tensor([0, 1])
text: ('멋진 작품.. 한가지 한가지 정성을 다해 만든 소품들..', '10년전엔 유쾌 상쾌 통쾌했던 영화.')
label: tensor([1, 1])
text: ('10점이 아깝지않은 다시봐도 명작', '어려운 영화....슬픈영화...가슴 아픈 이야기...')
label: tensor([1, 1])
text: ('10년전에 봤던 영화가 제 기억에 아직두 남아 있는영화....', '국산 페이크다큐가 갈 길은 아직 한참 먼 것 같다.'

label: tensor([1, 0])
text: ('기대 이상으로 재미있고 예쁘네요! 오버스러운 악역도 없고 ㅎㅎ', '이거 재밌게 봤는데 ^^')
label: tensor([1, 1])
text: ('영화관에 애들조차 웃지 않았다 ㅡㅡ', '더빙을 참...입장할때 메가폰줘야하는거아닌가요. 메가폰있으면 아이가더재미있어했을텐데아쉽네요.')
label: tensor([0, 0])
text: ('top 10 movie', '재밌구만뭘,,,')
label: tensor([1, 1])
text: ('재미없어', '한국공포는 여전함')
label: tensor([0, 0])
text: ('제의견은 60초 후에 공개됩니다 ㅎㅎ 아직도 남는 여운', '나만 이딴영화 볼수는 없음.. 새벽에 그냥 잘껄 괸히봣어..')
label: tensor([1, 1])
text: ('10점짜리 영화다. 시간이 지나도 다시 보고싶은 영화.', '정말이지 배우들 대사 한마디 한마디에 닭살 돋는다 대사가 왜 이렇게 쓰레기임? 작가님은 차라리 소설을 쓰세요 대본작가 말고')
label: tensor([1, 0])
text: ('로이킴,딕펑스,정준영 매력적이지 못했다 갈수록 흡입력이 왜 이리 떨어지지....', '얼굴이 망했당 ㅠㅠ')
label: tensor([0, 0])
text: ('허접한 여자괴수에 대해선 더이상 언급 않겠다.', '완전 재미없었네요...')
label: tensor([0, 0])
text: ('이게 뭐지???', '우와~ 정말 재미잇어요^^ 트레저 옐로,핑크 화이팅~!')
label: tensor([0, 1])
text: ('킬러들의 화끈한 한판! 8점 주고 싶지만 평점이 낮아 10점 줌.', '♥아 진짜 이런영화 사랑해줌ㅋㅋㅋ')
label: tensor([1, 1])
text: ('제목만강한응징자 내용은 소심한 할까..말까..소심해도너무소심함..보는사람이답답..', '달콤한 꿀과 톡쏘는 칠리의 만남')
label: tensor([0, 1])
text: ('스필버그 도 

text: ('평점 4.13도 많은 편이다..', '전재밌게보고있슴다 연기자분들화이팅')
label: tensor([0, 1])
text: ('너무 어수선하고 정신없고 이게뭐지?라는생각만든다 재밌게보던 프로였는데 이건 뭐..별반개도 아깝다 술자리에서 서로 떠드는듯한느낌..그나마 김구라씨가 정리라도안했음 방송인지 모를뻔', '재미 있어요.')
label: tensor([0, 1])
text: ('왼전재미있어요', '반장 나가고 하락세')
label: tensor([1, 0])
text: ('넘잼나^^^', '무언가를 생각하게 만들어준 영화')
label: tensor([1, 1])
text: ('good', '요즘 이런 영화 본적 있어!!')
label: tensor([0, 1])
text: ('코믹 호러용치고 수작인데 평점 너무 낮다', '요시모토 나라를 알고 있다면 보시길.')
label: tensor([1, 1])
text: ('한마디로 쓰레기영화다 스토리도 없고 왜 이렇게 카메라를 흔들어데는지 어지럽다 이 영화를 본다는것은 시간낭비이므로 말리고싶다', '사랑했던 여자가 생각난다 그 여자도 프랑스 영화를 좋아해서 인지...')
label: tensor([0, 1])
text: ('이건뭐야 ㅋㅋㅋ', 'pierre et gilles의 동성애적 사진영감에서 감독의 발상이 신선')
label: tensor([0, 1])
text: ('80년대 외화시리즈인기의 기폭이 되었던 명시리즈 다시보고싶네...ㅠㅠ', '많이 유치하다')
label: tensor([1, 0])
text: ('보는 내내 눈물이 멈추지 않았던 영화', '좋아요.')
label: tensor([1, 1])
text: ('정말 매력적인 영화였다 처음엔 거부감이 들었는데 갈수록 몰입되는 매력', '영국 및 미국의 연예계에 대한 지식이 있어야 함께 웃을 수 있다.')
label: tensor([1, 0])
text: ('예술적인 어두운 판타지 였다', '이런 영화도 있음..딴건 이해되나 여배우들 신음소

text: ('스턴트가 불쌍한 영화. 중간중간 피식 웃긴 했지만 영화가 전체적으로 허술하다.', '형사 결투자. 이 딴 영화를 찍은 불쌍한 하지원. 그래도 출연료 받았으니께 퉁 치세유.')
label: tensor([0, 0])
text: ('너무높다 깎아줘야되', '기막힌 시나리오와 개성만점 캐릭들과 영국식 발음이 즐겁게 해준다')
label: tensor([0, 1])
text: ('007의 미스터빈 버젼 .', '박중훈의 코믹연기 굿..!')
label: tensor([0, 1])
text: ('진짜 인간적으로 너무한다... 다른 예술영화에 비교해도 턱없이 부족한 상영관... ㅜㅜ', '진짜 재미있게 봤어요. 최곱니다!')
label: tensor([1, 1])
text: ('아무 생각 없이 봣는데 무지 재밋더라요', '이미 비공감 ....ㄱ\xad')
label: tensor([1, 0])
text: ('그냥 재미없음 지루함.', '제작비 말아 먹기 .. 참 쉽죠잉 .')
label: tensor([0, 0])
text: ('soso', '초등학생때 진짜 이거 잼나게봤었는데 초등학교 3학년대인가 50원주고보고했는데 ㅋ')
label: tensor([1, 1])
text: ('초절정 액션 활극... 이라면 별이 이렇게 많겠냐?', '전쟁나면부자들도거지가되는구나줄거리가끈기는것같네')
label: tensor([0, 0])
text: ('완전 사이코영화. 마지막은 더욱더 이 영화의질을 떨어트린다.', '지루하고 볼수록 짜증나는 스토리')
label: tensor([0, 0])
text: ('알 파치노의 멋진 연기, 브라이언 드 팔마의 멋진 연출. 굉장히 멋지고 감성적인 느와르', '이런 쓰레기도 영화라고~~')
label: tensor([1, 0])
text: ('도대체 무슨배짱으로 개봉한거냐????', '정말 멋진 배우들이 펼친 드라마')
label: tensor([0, 1])
text: ('전 재밌게 잘 봣어요..', '글자 10자 채우기도 귀찮네.')


...

text: ('내 인생 최고의 드라마로 기억될 것이다.', '토르 시리즈는 첫 단추를 잘못낀게 큰 화근인 듯...1편에서 되도 않는 러브라인 집어넣더니 여기선 여자가 중요한 무기로 취급당함ㅋ 우연한 설정에다가 건조한 럽라... 액션도 별로고...솔직히 8점은 아니다 진짜')
label: tensor([1, 0])
text: ('지금 나에게 과거로 돌아갈 수 있는 능력이 생기면 2시간 전으로 돌아가서 전편만한 속편 없다는말을 되새기며 마우스를 쿨릭하지 않았을 것', '맨마지막 레드포드가 자신의 보트가 불에 타오르는걸 보면서 물에 가라앉기 직전에 그 모습잠시 수영을 하며 체념한듯 그 모습잊혀지지가 않습니다')
label: tensor([0, 1])
text: ('도대체 어떤 스폰을 잡으면 나락까지 떨어져도 올려주냐?', '볼땐 몰렀는데 다른 코미디 영화와 비교해 보면 이보다 허접할수없다.')
label: tensor([0, 0])
text: ('극히 개인적이지만 시간이 아깝다 ㅠ_ㅠ', '20세기에 만들어졌어도 혹평을 받았을 3류스릴러. 시나리오, 연출, 편집, 구성, 캐릭터설정 뭐하나 맘에드는게 없다. 이런 영화를 왜수입하는지 정말 알수가없다,')
label: tensor([0, 0])
text: ('좀 촌스러울 수도 있겠지만 노래는 정말 좋아요...!! 스토리도 좋고...', '역시 강우석! 영화 그지같이 만드는건 여전하구나-')
label: tensor([1, 0])
text: ('아이 러브 크리시....', '감독은 의도도 안한거가지고 지들멋대로 해석하고 명작으로 치켜세움 ㅋㅋㅋ')
label: tensor([1, 0])
text: ('이런 영화를 이제서야 봤다니...조선의 소녀들이 떠올라 영화에서 헤어나오질 못하겠다. 어려운 소재를 정말 잘 풀어낸 영화다. 눈을 땔 수가 없고 정말 아름다운 영화.점수를 매기는게 예의가 아닌듯 하다. 배우들 및 모든 스텝들의 노고에 감사드린다.', '큰기대를 했지만 별로네요..')
label: tensor([1, 0])
t

label: tensor([0, 0])
text: ('주간아이돌 재밌어요!! 알랑가몰라도 재밌고(보미후니♥)금주의아이돌도 누구나올지 궁금하고 오래오래해요!!', '내 시간 ㅈㄴ 아깝네 ㅅㅂ 진짜')
label: tensor([1, 0])
text: ('몰입감 최고ㅋㅋ 조니에게 빙의되서 봤다 ㅋㅋㅋㅋ', '미유키역의 여배우빼고는 전부 진부하고 남자주인공 보다 그냥 짜증이 나버리네요..결국 영화가 더욱 싫어짐')
label: tensor([1, 0])
text: ('와....꼭보세요 꼬꼮꼭', '이런 영화를 위한 장르명을 새로 하나 지어줘야한다.죄와벌.만든 사람은 벌을 받아야함.이걸 보는니 차라리 귀여니소설을 목욕재개하고 정좌세로 정독을 하겠어요.보는 내내 내가 전생에 무슨 죄를 지었길래 이딴걸 보는데 시간을 써야하나?싶었다')
label: tensor([1, 0])
text: ('이런 세계적 오리지널 명공연을 이렇게 간접적으로라도 즐길수있게 해준 메가박스에 감사하며.. ㅋ실제같았음 장 마다 브레이크타임이잇엇을텐데 숼틈없이봐서 좀 숨막혔던거 빼고는.. 재밌었음^^ 자리도 정가운데였어서 몰입감이 꽤직접보는것처럼 느껴졌음^_^최고', '스토리엉성,생뚱맞고뜬금없는전개,베테랑연기자들의 진지한눈물연기가 아까웠던드라마')
label: tensor([1, 0])
text: ('초등학생때 우연히 엄마 가곡집에서 가사를 보고 비디오를 빌려 본 사운드 오브 뮤직은 고등학생때까지?일년에 두번씩 꼭 챙겨보는 인생영화가 되었다. 아름다운 노래소리와 연출들.. 영화의 모든 ost가 이렇게 좋을수 있을까? 볼때마다 소름이 돋는다', '2001년 한국 영화 생각하면, 이 작품은 지나치게 평가절하되어 있다.')
label: tensor([1, 1])
text: ("80년대 영화인줄 알았는데 '귀여운 여인'주제곡이 흘러나와 놀랐다", '이연걸 2명은 영화의 모든 장애물을 해결했다.')
label: tensor([0, 1])
text: ('아 아무것도 모르고 봤어야 했는데ㅠㅠㅠㅠ 너무 명작이라 그

text: ('너무 재미있게 본 영화', '정말 저질 .. 말그대로 저질 ... 내용두 없구 재미도 없구 돈만 날린 영화')
label: tensor([1, 0])
text: ('만점이다 ! ! !! 너무 재밌고 제시카알바언니 너무멋있다 ! !', '조금 느림. 여자 뻐꾸기')
label: tensor([1, 0])
text: ('너무나도 예쁘면서 슬픈 사랑을 보여준 영화', '내 삶을 다시 돌아보게 해준 영화')
label: tensor([1, 1])
text: ('ㅎㅎ 웃겨 호동님 연습하쇼', '여자의마음이구나. . .')
label: tensor([1, 1])
text: ('지금 고딩이 봐도 웃기다 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ', '승복이 나오는 영화가 그렇지 뭐')
label: tensor([1, 0])
text: ('굿 과학적인 결말 아주 맘에 드는듯 외계인이랑 붙어싸웟으면 더노잼 인정', '여전사가 나오는영화는 넘재밌는듯>< 내용도탄탄하고! 내용이근데넘어려움?..')
label: tensor([1, 1])
text: ('아이가 너무너무 조아해요 ㅋㅋㅋㅋ 또 보고싶다면 난리인지 ㅋㅋㅋㅋ한번 더 가야할듯싶어요~~', '영화계의 거장 피터잭슨이 만든영화중최악의영화인듯.데드얼라이브는정말잘만들엇는데')
label: tensor([1, 0])
text: ('B급이라도 스토리는 설득력 있게 했어야', '좋은 것은 모두 모아, 억지스런 해피 엔딩, 싫다.')
label: tensor([0, 0])
text: ('남자로 살지 여자로 살지 왜 내가 고민이 되지?', '재미없고 재미없고 개같네')
label: tensor([1, 0])
text: ('너무 쉽게 제시된 유토피아 그후 형식은 예상대로', 'epilogue2063 // 영화말고 님네 학교가 10점이져?')
label: tensor([0, 0])
text: ('재밌다고............난 정말 이상했음..ㅠㅠㅠ좀짜증남.', '다시 재개봉했으면 좋겠다..이렇게 묻히기엔 너무나 아까운 영화고 이렇게 잊혀지기엔 너무

label: tensor([0, 1])
text: ('아 영화 정말 더럽다 이딴 영화 보는 시간도 아깝네 밑에 평점 막주지 맙시다 거쫌...', '지겹다~')
label: tensor([0, 0])
text: ('1점도 아깝다 이게 15금이라는 자체가 말이 안되고 진짜 자극적인 요소만 끌어다놨음ㅋ 보고나서 기분 진심 더러워요 뭐 이게 영화야ㅋ 막장영화라는 장르가 생겼나ㅋ 돈 시간 다 아깝고 정말 영화 더러워요 미친;;', '별다른 cg나 특수 효과 없이 정말 재미지게 만듬,,,')
label: tensor([0, 1])
text: ('공감 안감. 여기서 젤 말많은 인간은 이발사', '헐 미친 ㅋㅋㅋㅋㅋㅋㅋ늑대의 유혹 만든 사람이 이것도 만들었었다니 ㅋㅋ내가 이거 동방신기때문에 봤지..ㅋㅋㅋost를 불렀다는 이유 하나만으로ㅋㅋㅋ')
label: tensor([0, 0])
text: ('스토리.과도한CG.. 나무랄때있는 졸작이다ㅋㅋ혹평에도 흥행에성공한 이유는뭘까? .. 배우들이름값도 못함ㅠㅠ', '편집 그런식으로 하지마요..제대로된 예능 내용도없고 감동도 와닿지도 않고...점점 안티팬이 늘어나게하는 프로그램인듯..작가님들 그만우려먹고 새로운 프로만들어요')
label: tensor([0, 0])
text: ('초등학교때 봤을때도 재미없었는데 지금은 재미뿐만 아니라 작품성까지 똥망인 게 보인다', '개핵노잼 보다끈건 처음이다 유치하고 인물들도 말도 안되고 뭔 과학자가 범죄자한테 로봇갔다바치고 범죄자도 착하고 결말도 유치하고 재미도 없고 의미도 없고 최악의 영화 씹노잼')
label: tensor([0, 0])
text: ('너무 오글거리고 어이없다..', '미묘한 심리묘사가 좋았어요')
label: tensor([0, 1])
text: ('처음엔 재밌게 봤거든요~ 근데 마지막까지 막무가내인 시어머니와 결단력없이 나약하게만 나오는 윤진이 역할도 정신을 놓은 것같은 선정이 역할에 이제는 질리네요~ 깔끔하게 유종의 미를 거두셨으면해요^^', '정말 끝내줫어')
label

label: tensor([0, 0])
text: ('컴퓨터는 나쁜것이 아니다 폭력적인 인간이 잘못한거지', '흐뭇하네요 공감 가는 부분이 많았습니다 마음에 들었어요^^')
label: tensor([1, 1])
text: ('시간 때우기에는 쓰레기 킬림타임용 한쿡영화보단 이게 낫다!', '말이필요없지.. 성룡영화에선.....무언가를다른것을느낄수있다')
label: tensor([1, 1])
text: ('죄송한데 이영화 어디서 구할수있죠?', '강동원씨 영화는 항상 극장에서 보는데 이번 만큼은 정말...인터뷰 보니까 강동원씨는 송혜교씨가 먼저 캐스팅 되고 서로 안부 물어보다가 이 영화 얘길 접하고 시나리오 건네받아서 출연하게 됐다던데 송혜교씨에게 안부를 물어봤다가 영화인생에 오점을 남겼네요')
label: tensor([1, 0])
text: ('전작보다 못한 속편. 그리고 끝까지보는데 이렇게힘드나..', '토퍼 보는 맛에 겨우 봤다')
label: tensor([0, 0])
text: ('요샌 네이버 평점도 믿을게 못되는군. 행동거지가 예측이될정도니 한심하고. 온갖 바보같은 짓은 다하고 ㅉㅉㅉ평점 8점이라 얼른 봤더니 3명 평가에 다 30대냐...0점이 없는게 안타깝다.', '김기덕에겐 있지만 이 작품엔 없는것 - 그것은 연기, 연출, 배우')
label: tensor([0, 0])
text: ('가지가지하는 정말 못생기고 매력이라곤 단 하나 찾을 수 없는 여자를 약 110분 동안이나 구경할 수 있는 유일한 영화.', '흥미로운 설정이지만..')
label: tensor([0, 0])
text: ('결말에서의 아쉬움을 뺀다면 거의 퍼펙트', '5점씩이나 받다니')
label: tensor([1, 0])
text: ('박보영 때문이 아니더라도 이영화는 좋음..', '피만 가득한 더럽고 잔인한 막장 쓰레기 영화. 장르는 무슨... 끝까지 본 내가 한심하다.')
label: tensor([1, 0])
text: ('공포스릴러의목적에서벗어나 어떤의미에선수작일지몰라

In [ ]:
#working properly but recomment using google colab!

optimizer = Adam(model.parameters(), lr=1e-6)

itr = 1
p_itr = 500
epochs = 1
total_loss = 0
total_len = 0
total_correct = 0

model.train()

for epoch in range(epochs):
    
    for text, label in train_loader:
        optimizer.zero_grad()
        
        # encoding and zero padding
        encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
        padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
        
        sample = torch.tensor(padded_list)
        sample, label = sample.to(device), label.to(device)
        labels = torch.tensor(label)
        outputs = model(sample, labels=labels)
        loss, logits = outputs

        pred = torch.argmax(F.softmax(logits), dim=1)
        correct = pred.eq(labels)
        total_correct += correct.sum().item()
        total_len += len(labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        
        if itr % p_itr == 0:
            print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
            total_loss = 0
            total_len = 0
            total_correct = 0

        itr+=1


In [ ]:
# evaluation
model.eval()

nsmc_eval_dataset = NsmcDataset(test_df)
eval_loader = DataLoader(nsmc_eval_dataset, batch_size=2, shuffle=False, num_workers=2)

total_loss = 0
total_len = 0
total_correct = 0

for text, label in eval_loader:
    encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
    padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
    sample = torch.tensor(padded_list)
    sample, label = sample.to(device), label.to(device)
    labels = torch.tensor(label)
    outputs = model(sample, labels=labels)
    _, logits = outputs

    pred = torch.argmax(F.softmax(logits), dim=1)
    correct = pred.eq(labels)
    total_correct += correct.sum().item()
    total_len += len(labels)

print('Test accuracy: ', total_correct / total_len)
 